# Fetch Rewards Coding Exercise - Data Analyst


#### AUTHOR : Viraj Shukla

#### DATE CREATED : (Saturday) 2nd October 2021

#### DESC: 
In this python notebook I will be addressing following requirement and processing the given Json data into structured data.

#### What are the requirements?
1.Review unstructured JSON data and diagram a new structured relational data model

2.Generate a query that answers a predetermined business question

3.Generate a query to capture data quality issues against the new structured relational data model

4.Write a short email or Slack message to the business stakeholder


## 1. Review unstructured JSON data and diagram a new structured relational data model
#### First step is process and clean the nested JSON data into simplified structured data and to understand it for data modeling

In [541]:
#importing famous pandas library for processing data and datetime for date conversion
import pandas as pd
from datetime import datetime

#### JSON data is provided in 3 zip files (receipts,users, brands), firstly I extracted the zip files and loaded the JSON files using the following pandas function.

In [542]:
receipts = pd.read_json('receipts.json',lines=True)
receipts.info()
receipts.head(3)
#using info(), head() to understand data format,  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
_id                        1119 non-null object
bonusPointsEarned          544 non-null float64
bonusPointsEarnedReason    544 non-null object
createDate                 1119 non-null object
dateScanned                1119 non-null object
finishedDate               568 non-null object
modifyDate                 1119 non-null object
pointsAwardedDate          537 non-null object
pointsEarned               609 non-null float64
purchaseDate               671 non-null object
purchasedItemCount         635 non-null float64
rewardsReceiptItemList     679 non-null object
rewardsReceiptStatus       1119 non-null object
totalSpent                 684 non-null float64
userId                     1119 non-null object
dtypes: float64(4), object(11)
memory usage: 131.2+ KB


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '26.00', 'itemPrice': '26.00', 'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'quantityPurchased': 5, 'userFlaggedBarcode': '4011', 'userFlaggedNewItem': True, 'userFlaggedPrice': '26.00', 'userFlaggedQuantity': 5}]",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'partnerItemId': '1', 'quantityPurchased': 1}, {'barcode': '028400642255', 'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'finalPrice': '10.00', 'itemPrice': '10.00', 'needsFetchReview': True, 'needsFetchReviewReason': 'USER_FLAGGED', 'partnerItemId': '2', 'pointsNotAwardedReason': 'Action not allowed for user and CPG', 'pointsPayerId': '5332f5fbe4b03c9a25efd0ba', 'preventTargetGapPoints': True, 'quantityPurchased': 1, 'rewardsGroup': 'DORITOS SPICY SWEET CHILI SINGLE SERVE', 'rewardsProductPartnerId': '5332f5fbe4b03c9a25efd0ba', 'userFlaggedBarcode': '028400642255', 'userFlaggedDescription': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'userFlaggedNewItem': True, 'userFlaggedPrice': '10.00', 'userFlaggedQuantity': 1}]",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'userFlaggedBarcode': '4011', 'userFlaggedNewItem': True, 'userFlaggedPrice': '26.00', 'userFlaggedQuantity': 3}]",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


#### After understanding the given nested receipt data, I created a general date conversion function, since date provided is in utc format. Assuming fetch rewards servers use UTC format for date. 

In [543]:
def date_converter(x):
    '''
    Date provided is in UTC Format and we use this function to convert it to understandable format
    
    INPUTS
        x : Date string in UTC Format
    
    OUTPUTS
        y : Return Date in datetime format
    
    '''
    
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)

#### In the following code I am extracting values (JSON key-value format) from nested dictionaries of receipts data. I analyzed receipts data online (JSON data Viewer) to get Keys of the below nested columns. example receipts 'id' has 'oid' as nested Key and we are only interested in the value of this Key, i.e. '5ff1e1eb0a720f0523000575'. Similarly, processed the remaining date colums for its values and then converted UTC dates using above functions.

In [544]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_converter(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_converter(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_converter(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_converter(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_converter(x))

In [545]:
#head() gives us a view of cleaned and structured data until 'rewardsReceiptItemList' column
receipts.head(2)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '26.00', 'itemPrice': '26.00', 'needsFetchReview': False, 'partnerItemId': '1', 'preventTargetGapPoints': True, 'quantityPurchased': 5, 'userFlaggedBarcode': '4011', 'userFlaggedNewItem': True, 'userFlaggedPrice': '26.00', 'userFlaggedQuantity': 5}]",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT FOUND', 'finalPrice': '1', 'itemPrice': '1', 'partnerItemId': '1', 'quantityPurchased': 1}, {'barcode': '028400642255', 'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'finalPrice': '10.00', 'itemPrice': '10.00', 'needsFetchReview': True, 'needsFetchReviewReason': 'USER_FLAGGED', 'partnerItemId': '2', 'pointsNotAwardedReason': 'Action not allowed for user and CPG', 'pointsPayerId': '5332f5fbe4b03c9a25efd0ba', 'preventTargetGapPoints': True, 'quantityPurchased': 1, 'rewardsGroup': 'DORITOS SPICY SWEET CHILI SINGLE SERVE', 'rewardsProductPartnerId': '5332f5fbe4b03c9a25efd0ba', 'userFlaggedBarcode': '028400642255', 'userFlaggedDescription': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', 'userFlaggedNewItem': True, 'userFlaggedPrice': '10.00', 'userFlaggedQuantity': 1}]",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


In [546]:
#for loop for extracting rewardsReceiptItemList column out of receipts table
l = []
for i in range(len(receipts)):
    try:
        l.append(receipts['rewardsReceiptItemList'].iloc[i][0])
    except:
        #print(receipts['rewardsReceiptItemList'].iloc[i])
        pass



#### Now its time for 'rewardsReceiptItemList' column cleanup, when I pulled receipts data in the online Json screener, I figured 'rewardsReceiptItemList' column had about 18 columns within it. So I decided to pull the column out of receipts table to get a simplified table for data modeling. I am aware I could have extracted these 18 columns and again merged it back to receipts table, but for a simplified data model I choose to create a new table 'rewards_receipts_item' out of 'rewardsReceiptItemList' column.


In [547]:
#From my understanding "rewardsReceiptStatus" column better suits 'rewards_receipts_item' table, so added it
rewards_receipts_item = pd.DataFrame(l)
rewards_receipts_item["rewardsReceiptStatus"] = receipts["rewardsReceiptStatus"]
rewards_receipts_item['receipts_id']= receipts["_id"] # copied receipts ID to the new table for joining and as there is 1 to many relationship between the table
rewards_receipts_item.sample()

,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,finalPrice,itemNumber,itemPrice,...,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,rewardsReceiptStatus,receipts_id
204,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,1,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FINISHED,5ffc8fa10a7214adca00002e


In [548]:
# Generating unique identifier for 'rewards_receipts_item' table
import numpy as np
rewards_receipts_item['receipts_item_id'] = np.zeros((len(rewards_receipts_item),1))
for i in range(len(rewards_receipts_item)):
    rewards_receipts_item['receipts_item_id'].iloc[i]= i+1

C:\Users\viraj\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [549]:
# Just moving ID's to the front of the 'rewards_receipts_item' table
col_val = ['rewards_id','receipts_id','rewardsReceiptStatus',
           'barcode', 'brandCode', 'competitiveProduct', 'competitorRewardsGroup',
       'deleted', 'description', 'discountedItemPrice', 'finalPrice',
       'itemNumber', 'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity']
rewards_receipts_item = rewards_receipts[col_val]
rewards_receipts_item.head()

,rewards_id,receipts_id,rewardsReceiptStatus,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,...,priceAfterCoupon,quantityPurchased,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
0,1.0,5ff1e1eb0a720f0523000575,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,5.0,NaN,NaN,NaN,4011,NaN,True,26.00,5.0
1,2.0,5ff1e1bb0a720f052300056b,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,5ff1e1f10a720f052300057a,REJECTED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4011,NaN,True,26.00,3.0
3,4.0,5ff1e1ee0a7214ada100056f,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,4.0,NaN,NaN,NaN,4011,NaN,True,28.00,4.0
4,5.0,5ff1e1d20a7214ada1000561,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [550]:
# Now we have a new table 'rewards_receipts_item' for 'rewardsReceiptItemList', we can get rid of the nested column
receipts=receipts.drop(['rewardsReceiptItemList'], axis=1)

In [551]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
_id                        1119 non-null object
bonusPointsEarned          544 non-null float64
bonusPointsEarnedReason    544 non-null object
createDate                 1119 non-null object
dateScanned                1119 non-null object
finishedDate               568 non-null object
modifyDate                 1119 non-null object
pointsAwardedDate          537 non-null object
pointsEarned               609 non-null float64
purchaseDate               671 non-null object
purchasedItemCount         635 non-null float64
rewardsReceiptStatus       1119 non-null object
totalSpent                 684 non-null float64
userId                     1119 non-null object
dtypes: float64(4), object(10)
memory usage: 122.5+ KB


In [552]:
# Converting the dataframe to .csv files, as I planned to load them into SQL Server for faster querings and Modeling
receipts.to_csv('receiptstosql.csv')

In [553]:
# Converting the dataframe to .csv files, as I planned to load them into SQL Server for faster querings and Modeling
rewards_receipts_item.to_csv('rewards_receiptstosql.csv')

## Users Table Processing (Following above steps for cleaning)

In [554]:
#users processing
#Similarly loading users data using pandas and understand it using info() & head()
users = pd.read_json('users.json',lines=True)
users.info()
users.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
_id             495 non-null object
active          495 non-null bool
createdDate     495 non-null object
lastLogin       433 non-null object
role            495 non-null object
signUpSource    447 non-null object
state           439 non-null object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [555]:
# Since not all lastlogin have date, we will have to drop 'NA'
set().union(*(d.keys() for d in users.dropna(subset=['lastLogin'])['lastLogin']))

{'$date'}

In [556]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

In [557]:
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [558]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [559]:
users.to_csv('userstosql.csv')

## Brands Table Processing (Following above steps for cleaning)

In [512]:
brands = pd.read_json('brands.json',lines=True)
brands.info()
brands.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
_id             1167 non-null object
barcode         1167 non-null int64
brandCode       933 non-null object
category        1012 non-null object
categoryCode    517 non-null object
cpg             1167 non-null object
name            1167 non-null object
topBrand        555 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 73.0+ KB


,_id,barcode,brandCode,category,categoryCode,cpg,name,topBrand
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,NaN,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}",test brand @1612366101024,0.0
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,STARBUCKS,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}",Starbucks,0.0
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146176,0.0


In [513]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])
brands['cpg_id']=brands['cpg'].apply(lambda x: x['$id']['$oid'])
brands['cpg_ref']=brands['cpg'].apply(lambda x: x['$ref'])

In [434]:
brands.head(1)

,_id,barcode,brandCode,category,categoryCode,cpg,name,topBrand,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}",test brand @1612366101024,0.0,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, '$ref': 'Cogs'}",Starbucks,0.0,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146176,0.0,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, '$ref': 'Cogs'}",test brand @1612366146051,0.0,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, '$ref': 'Cogs'}",test brand @1612366146827,0.0,5332fa12e4b03c9a25efd1e7,Cogs


In [514]:
brands=brands.drop(['cpg'], axis=1)


In [436]:
brands.head()

,_id,barcode,brandCode,category,categoryCode,name,topBrand,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,NaN,Baking,BAKING,test brand @1612366101024,0.0,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,Starbucks,0.0,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,test brand @1612366146176,0.0,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,test brand @1612366146051,0.0,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,5332fa12e4b03c9a25efd1e7,Cogs


In [295]:
brands_new.to_csv('brandstosql.csv')

#### Reviewing final data frames for SQL processing 

In [441]:
receipts.sample()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
434,600ed4820a720f0535000015,750.0,"Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)",2021-01-25 14:24:02,2021-01-25 14:24:02,2021-01-25 14:24:02,2021-01-25 14:24:02,2021-01-25 14:24:02,750.0,2021-01-24 14:24:02,1.0,FINISHED,1.0,600ed4815edb7811cde6b50b


In [516]:
rewards_receipts_item.sample()

,rewards_id,receipts_id,rewardsReceiptStatus,barcode,brandCode,competitiveProduct,competitorRewardsGroup,deleted,description,discountedItemPrice,...,priceAfterCoupon,quantityPurchased,rewardsGroup,rewardsProductPartnerId,targetPrice,userFlaggedBarcode,userFlaggedDescription,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity
136,137.0,5ff74ff50a720f052300061e,FINISHED,4011,NaN,NaN,NaN,NaN,ITEM NOT FOUND,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [439]:
users.sample()

,_id,active,createdDate,lastLogin,role,signUpSource,state
463,5fa41775898c7a11a6bcef3e,True,2020-11-05 15:17:09,2021-03-04 16:02:02,fetch-staff,Email,NaN


In [438]:
brands.sample()

,_id,barcode,brandCode,category,categoryCode,name,topBrand,cpg_id,cpg_ref
909,60074261be37ce360be639f6,511111319795,nan,Baking,BAKING,test brand @1611088480964,0.0,60074260be37ce360be639f5,Cogs


## 2. Generate a query that answers a predetermined business question?

#### I will be answersing predetermined business question using SQL ALCHEMY in the python notebook, as I wanted to have major answers in one place. I will also provide a seperate SQL query file as I have loaded these data frame to SQL server DB

In [380]:
from sqlalchemy import create_engine

In [381]:
engine = create_engine('sqlite://', echo=False)

#### Firstly Users should be distinct , so removing duplicate users, and before quering the data, loading tables to the SQL Engine

In [382]:
users.drop_duplicates(subset=['_id']).to_sql('users',con=engine)

In [383]:
# replacing periods to underscore
receipts.columns = receipts.columns.str.replace(".", "_")

In [384]:
receipts.to_sql('receipts',con=engine)

In [385]:
brands.drop_duplicates(subset=['brandCode']).to_sql('brands',con=engine)

In [386]:
rewards_receipts_item.to_sql('rewards_receipts_item',con=engine)

### 2A. What are the top 5 brands by receipts scanned for most recent month?

In [474]:
# To get top 5 brands, I will be using CTE. 
# I have created 3 CTE as my data Model needs to get 'brands' table data via 'rewards_receipts_item' table and scanned date from receipts table

engine.execute("with brands_cte as (\
            select distinct name, brandCode from brands\
              ),\
                items as (\
                select receipts_id, brandcode from rewards_receipts_item\
                ),\
                receipts_cte as (\
                select _id as receiptid, strftime('%Y-%m', dateScanned) as date from receipts\
                ) \
                select r.date, b.name, count(r.receiptid) as cnt_receiptid from receipts_cte as r\
                inner join items as i on i.receipts_id=r.receiptid\
                inner join brands as b on i.brandCode=b.brandcode\
                group by 1,2\
                having r.date = '2021-01'\
                order by 3 desc limit 5\
               ").fetchall() 
#In the above query I have used explicit dates in having clause because, most recent month is march 2021 and it doesn't have any brand data, so the most recent records available in the data are for january 2021 
# the following having clause will automate the most recent month data, having r.date in (select distinct max(date) from receipts_cte)

[('2021-01', 'Pepsi', 4),
 ('2021-01', 'KNORR', 3),
 ('2021-01', 'Huggies', 2),
 ('2021-01', 'Doritos', 1),
 ('2021-01', 'KLONDIKE', 1)]

###  2B. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [481]:
# To get average spending, we will need 'totalspent' and 'rewardsReceiptStatus' to know which status is greater, 
# Used CASE WHEN conditioning to print 'ACCEPTED' instead of 'Finished'
engine.execute("select case when rewardsReceiptStatus='FINISHED' then 'ACCEPTED' else 'REJECTED' end\
               from receipts\
               where rewardsReceiptStatus in ('FINISHED','REJECTED')\
               group by rewardsReceiptStatus\
               order by avg(totalspent) desc\
               limit 1\
               ").fetchall()

[('ACCEPTED',)]

### 2C. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [489]:
# using the same above query logic here for purchased item
engine.execute("select\ 
                   case\ 
                       when rewardsReceiptStatus='FINISHED' then 'ACCEPTED'\ 
                       else 'REJECTED'\ 
                    end\
               from receipts\
               where rewardsReceiptStatus in ('FINISHED','REJECTED')\
               group by rewardsReceiptStatus\
               order by sum(purchasedItemCount) desc\
               limit 1\
               ").fetchall()

[('ACCEPTED',)]

### 2D. Which brand has the most spend among users who were created within the past 6 months?

In [500]:
# Again I will be creating CTE for getting most amount spend by user created in past 6 months
# Created User CTE and filted on datecreated of users, and then joing with Item table
engine.execute("with users_cte as (\
              select distinct _id as userid from users\
              where createddate >= (select date(max(createddate), '-6 month') from users)\
              ),\
               receipts_cte as (\
               select userid, totalspent, _id as r_id from receipts\
               ),\
               items_cte as (\
               select brandcode, receipts_id from rewards_receipts_item\
               )\
               select i.brandcode, sum(r.totalspent)\
               from items_cte as i\
               join receipts_cte r on i.receipts_id=r.r_id\
               where r.userid in (select userid from users_cte)\
               and i.brandcode is not null\
               group by 1 order by 2 desc limit 1\
               ").fetchall()

[('KNORR', 622.94)]

# 3. Generate a query to capture data quality issues against the new structured relational data model

### Data Quality Issue

#### Check Duplicate Users: There should be distinct users, as one user can have many receipts (one to many cardinality). But the other way round is not possible (many receipts can't have many user)

In [490]:
users.sample()

,_id,active,createdDate,lastLogin,role,signUpSource,state
435,5fc961c3b8cfca11a077dd33,True,2020-12-03 22:08:03,2021-02-26 22:39:16,fetch-staff,Email,NH


#### In the code below I have found, how many dulicate users present in the data and also checked for ID column

In [302]:
users_after_all_columns_duplicate_removal = users.drop_duplicates()

In [305]:
unique_users, total_users = (len(users_after_all_columns_duplicate_removal),len(users))

In [535]:
total_users


1167

In [306]:
print('Total Number of Duplicate(All the columns) Users:',total_users-unique_users)

Total Number of Duplicate(All the columns) Users: 283


In [307]:
users_after_all_ids_duplicate_removal = users['_id'].drop_duplicates()

In [308]:
unique_users, total_users = (len(users_after_all_ids_duplicate_removal),len(users))

In [309]:
print('Total Number of Duplicate(IDS) Users:',total_users-unique_users)

Total Number of Duplicate(IDS) Users: 283


#### Check If we have missing prices in ReceiptsItemList: Below I have checked for prices missing in Item column and comparing them to FinalPrice.

In [517]:
rewards_receipts_item.columns

Index(['rewards_id', 'receipts_id', 'rewardsReceiptStatus', 'barcode',
       'brandCode', 'competitiveProduct', 'competitorRewardsGroup', 'deleted',
       'description', 'discountedItemPrice', 'finalPrice', 'itemNumber',
       'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity'],
      dtype='object')

In [518]:
rewards_receipts_item[['itemPrice','targetPrice','finalPrice']].head()

,itemPrice,targetPrice,finalPrice
0,26.00,NaN,26.00
1,1,NaN,1
2,NaN,NaN,NaN
3,28.00,NaN,28.00
4,1,NaN,1


In [520]:
# Here we find out that itemPrice and FinalPrice are identical and there is no rewards for any Item 
(rewards_receipts_item['itemPrice'].astype(float) - rewards_receipts_item['finalPrice'].astype(float)).dropna().sum()

0.0

This shows that the columns 'itemPrice' and 'finalPrice' have exactly same data. Lets check if we have any price set to Nan

In [521]:
rewards_receipts_item['finalPrice'].isna().sum()

14

In [522]:
print('Out of ',len(rewards_receipts_item),'Samples,',rewards_receipts_item['finalPrice'].isna().sum(),'samples have NaN as Final Price')

Out of  679 Samples, 14 samples have NaN as Final Price


### Checked if we have missing total spent amount from receipts

In [524]:
receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId'],
      dtype='object')

In [525]:
print('Out of ',len(receipts),'Samples,',receipts['totalSpent'].isna().sum(),'samples have NaN as totalSpent')

Out of  1119 Samples, 435 samples have NaN as totalSpent


### Checked if we have unique barcodes

In [526]:
brands.columns

Index(['_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name',
       'topBrand', 'cpg_id', 'cpg_ref'],
      dtype='object')

Bar Code of all the products must be unique

In [340]:
brands_temp = brands[brands['barcode'].notna()]

In [342]:
brands_removing_duplicate_barcodes =brands_temp['barcode'].drop_duplicates()

In [343]:
unique_users, total_users = (len(brands_removing_duplicate_barcodes),len(brands_temp))

In [344]:
print('Total Number of Duplicate BarCodes:',total_users-unique_users)

Total Number of Duplicate BarCodes: 7


We have 7 Duplicate Barcodes, which might cause an issue


### Mactching Barcode and brandcode from Brands to that of  rewards_receipts_item

In [536]:
rewards_receipts_item.columns

Index(['rewards_id', 'receipts_id', 'rewardsReceiptStatus', 'barcode',
       'brandCode', 'competitiveProduct', 'competitorRewardsGroup', 'deleted',
       'description', 'discountedItemPrice', 'finalPrice', 'itemNumber',
       'itemPrice', 'metabriteCampaignId', 'needsFetchReview',
       'needsFetchReviewReason', 'originalMetaBriteBarcode',
       'originalMetaBriteDescription', 'originalMetaBriteQuantityPurchased',
       'originalReceiptItemText', 'partnerItemId', 'pointsEarned',
       'pointsNotAwardedReason', 'pointsPayerId', 'preventTargetGapPoints',
       'priceAfterCoupon', 'quantityPurchased', 'rewardsGroup',
       'rewardsProductPartnerId', 'targetPrice', 'userFlaggedBarcode',
       'userFlaggedDescription', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity'],
      dtype='object')

In [537]:
brands.columns

Index(['_id', 'barcode', 'brandCode', 'category', 'categoryCode', 'name',
       'topBrand', 'cpg_id', 'cpg_ref'],
      dtype='object')

In [538]:
receipts.columns

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptStatus', 'totalSpent', 'userId'],
      dtype='object')

In [530]:
# replace brandcode with barcode to get comparison between brandcode and barcode
uniques_barcode_or_brandcode_in_receipts_items = set(rewards_receipts_item['brandCode'].dropna())
uniques_barcodes_or_brandcode_in_brands = set(brands['brandCode'].dropna())

In [531]:
intersection = uniques_barcode_or_brandcode_in_receipts_items.intersection(uniques_barcodes_or_brandcode_in_brands)
a = len(uniques_barcode_or_brandcode_in_receipts_items - intersection)

In [532]:
b = len(uniques_barcode_or_brandcode_in_receipts_items)

In [533]:
len(uniques_barcodes_or_brandcode_in_brands)

1156

In [534]:
print('Out of',str(b),'samples,',str(a),'smaples do naot have a match in the brands table')

Out of 18 samples, 16 smaples do naot have a match in the brands table


Also, the barcodes present in the table 'rewards_receipts_item' do not have a single match with barcodes in table 'brands'. Meaning they are identical of each brands.